In [1]:
# 파일이름 : MES_FMB_data_generator.ipynb
# 코드설명 : 고무 배합비와 제조실행시스템(MES)에서 얻어진 검사 CMB 연결 된 FMB 데이터에 대한 전처리 및 모델 학습 데이터 세트 생성
# 입/출력 : MES 통합 데이터 (DA측 제공) / MES FMB 데이터 세트(17개)
# 유의 사항 : 물성값이 0인 경우 제외
# 최종수정 : 2023년 11월 27일
# 제 작 자 : 맹영준 (myj6223@micube.co.kr), 홍민성 (mshong@micube.co.kr)
# Copyright : MICUBE Solution, Inc

In [2]:
import seaborn as sns
import matplotlib.font_manager as fm

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
import warnings
%matplotlib inline

warnings.filterwarnings(action='ignore')

pd.set_option("display.max_columns", 500)
pd.set_option('display.max_rows', 50)

import os
import matplotlib as mpl

## 1. 데이터 로드 및 데이터 분리
- CMB와 연계된 FMB 데이터 필터링

In [63]:
df = pd.read_csv('./dataset/sfites_real_mix_data_full.csv')

# "1_JRCODE" 컬럼의 마지막 문자가 "C1"인 행만 필터링
c1_rows = df['RUBBER'].str.endswith('C1', na=False)
c1_filtered_df = df[c1_rows]

# f!만
non_c1_filtered_df = df[~c1_rows]

# f1중에서 c1필요없는것
c1_rows_2 = non_c1_filtered_df['1_JRCODE'].str.endswith('C1', na=False)
non_c1_f1 = non_c1_filtered_df[~c1_rows_2]

# f1중에서 c1필요한 것
c1_filtered_df_2 = non_c1_filtered_df[c1_rows_2]

## 2.전처리 시작

### 2.1 FMB, CMB 데이터 기초 전처리
- 통계적 이상치 제거
- 변수 특성별 대푯값 추출

#### 2.1.1) CMB와 연계된 FMB 데이터에 대한 통계적 이상치 제거
- FMB별 10개 미만 CMB와 연계된 경우 제거
- 필터링된 고무 배합별로 수치형 변수에 Boxplot 민감도 3으로 이상치 제거

In [5]:
%%time

# 러버 그룹별로 그룹내의 요소 개수가 10이상인 것을 찾고, iqr로 이상치 제거

# "1_JRCODE" 컬럼의 마지막 문자가 "C1"인 행만 필터링
c1_rows = df['1_JRCODE'].str.endswith('C1', na=False)
c1_filtered_df = df[c1_rows]

# 원본 데이터프레임(df)에서 "RUBBER" 값이 c1_filtered_df의 "" 값과 일치하는 행만 선택
matching_rows = df[df['RUBBER'].isin(c1_filtered_df['1_JRCODE'])]

# "RUBBER" 컬럼의 값이 같은 행만 필터링
rubber_filtered_df = matching_rows[matching_rows.duplicated('RUBBER', keep=False)]

# 10개 이상의 데이터를 가진 그룹만 필터링
group_counts = matching_rows['RUBBER'].value_counts()
valid_groups = group_counts[group_counts >= 10].index
valid_df = matching_rows[matching_rows['RUBBER'].isin(valid_groups)]

# 숫자형 변수만 선택
selected_cols = valid_df.filter(regex='RPM[1-9]|RPM10|TEMP[1-9]|TEMP[1-2][0-9]|TIME[1-9]|TIME[1-2][0-9]|JUK[1-9]|JUK[1-2][0-9]|[1-9]_PHR|[1-2][0-9]_PHR')
numeric_cols = selected_cols.columns

# IQR을 이용한 이상치 제거 함수
def remove_outliers(group):
    for col in numeric_cols:
#         print(group[col].isna().sum(), len(group))
        if group[col].isna().sum() == len(group):
            continue
        else:
            Q1 = group[col].quantile(0.25)
            Q3 = group[col].quantile(0.75)
            IQR = Q3 - Q1
            group = group[(group[col] >= (Q1 - 3 * IQR)) & (group[col] <= (Q3 + 3 * IQR))]
    return group

# 그룹별로 이상치 제거 적용
filtered_df = valid_df.groupby('RUBBER').apply(remove_outliers).reset_index(drop=True)

CPU times: user 1min 34s, sys: 124 ms, total: 1min 34s
Wall time: 1min 34s


#### 2.1.2) FMB와 연계된 CMB데이터에서 변수 특성별 대푯값 추출

##### 이상치가 제거된 FMB와 연계된 CMB데이터에서 공정조건들 평균값 계산 (재료코드 제외)
- 수치형 변수로 한정 (예: 공정조건, 검사조건)

In [6]:
# 'RUBBER' 컬럼을 기준으로 각 컬럼의 평균값 계산
grouped_df = filtered_df.groupby('RUBBER').mean(numeric_only=True)
grouped_df.reset_index(inplace=True)


# CMB데이터가 필요한 FMB 데이터 선택
matching_rows_df = c1_filtered_df_2[c1_filtered_df_2['1_JRCODE'].isin(grouped_df['RUBBER'])]

pro_cols = [f'RPM{i}' for i in np.arange(1,21)] + [f'TEMP{i}' for i in np.arange(1,21)] + [f'TIME{i}' for i in np.arange(1,21)] + [f'JUK{i}' for i in np.arange(1,21)]
#jrcode_cols = [f'{i}_JRCODE' for i in np.arange(1,22)] + [f'{i}_PHR' for i in np.arange(1,22)] + [f'{i}_PUTGB' for i in np.arange(1,22)]
jrcode_cols = [f'{i}_PHR' for i in np.arange(1,22)] + [f'{i}_PUTGB' for i in np.arange(1,22)]

exce = [f'{i}_JRCODE' for i in np.arange(1,22)] + list(matching_rows_df[['LOTNO','RUBBER','MECHCD','HS','HS_RESULT','SG','SG_RESULT','TS','TS_RESULT','EB','EB_RESULT','MNY','MNY_RESULT','REHO_MIN','REHO_MAX','REHO_TS2','REHO_TC90','REHO_RESULT','SCR','SCR_RESULT','HS_TIME','HS_TEMP','SG_TIME','SG_TEMP','TS_TIME','TS_TEMP','EB_TIME','EB_TEMP','MNY_TIME','MNY_TEMP','REHO_TIME','REHO_TEMP','SCR_TIME','SCR_TEMP']].columns)

##### 이상치가 제거된 CMB데이터에서 JRCODE 추출
- 러버별로 중복된 재료 코드들이 존재하여 하나씩만 선택

In [9]:
grouped_filtered_df = filtered_df.groupby('RUBBER').apply(
    lambda x: x[['RUBBER'] + [col for col in x.columns if 'JRCODE' in col]].head(1)
).reset_index(drop=True)

##### 앞서 전처리된 CMB 데이터의 공정조건+배합비 데이터를 CMB와 연계된 FMB 데이터와 Merge

In [10]:
# grouped_df -> C1 평균값 계산 데이터 셋
# matching_rows_df -> C1이 필요한 F1데이터 셋
# grouped_filtered_df -> 전처리된 C1데이터의 재료코드 데이터셋

merged_df = pd.merge(matching_rows_df, grouped_df.add_suffix('_c'), how='inner', left_on='1_JRCODE', right_on='RUBBER_c')

# 병합 후 필요한 컬럼만 선택
final_cols_from_matching = [col for col in exce if col in merged_df.columns]
final_cols_from_grouped = [f"{col}_c" for col in pro_cols + jrcode_cols if f"{col}_c" in merged_df.columns]

# 최종 선택할 컬럼 리스트 (matching_rows_df와 grouped_df에서 가져온 컬럼 모두 포함)
final_cols = final_cols_from_matching + pro_cols + jrcode_cols + final_cols_from_grouped

final_df = merged_df[final_cols]
final_df = pd.merge(final_df, grouped_filtered_df.add_suffix('_c'), how='left', left_on='1_JRCODE', right_on='RUBBER_c')

### 2.2 리스트 데이터 전처리
- NaN값 제거
- 0값 제거
- 패딩 작업

데이터프레임으로 되어있는 CMB+FMB 데이터의 중간에 NaN값과 CMB/FMB 그룹별,특성 컬럼별 끝단에 연속되는 0값이 있음.   
이를 쉽게 제거하기 위해 리스트로 변환하여 아래 단계와 같이 전처리 작업을 수행함.

In [ ]:
# 리스트 특성별 그룹화 작업

columns = ['1_JRCODE', '2_JRCODE', '3_JRCODE', '4_JRCODE', '5_JRCODE', '6_JRCODE', '7_JRCODE', '8_JRCODE', '9_JRCODE', '10_JRCODE',
           '11_JRCODE', '12_JRCODE', '13_JRCODE', '14_JRCODE', '15_JRCODE', '16_JRCODE', '17_JRCODE', '18_JRCODE', '19_JRCODE', '20_JRCODE',
           '21_JRCODE', 'LOTNO', 'RUBBER', 'MECHCD', 'HS', 'HS_RESULT', 'SG', 'SG_RESULT', 'TS', 'TS_RESULT', 'EB', 'EB_RESULT', 'MNY',
           'MNY_RESULT', 'REHO_MIN', 'REHO_MAX', 'REHO_TS2', 'REHO_TC90', 'REHO_RESULT', 'SCR', 'SCR_RESULT', 'HS_TIME', 'HS_TEMP',
           'SG_TIME', 'SG_TEMP', 'TS_TIME', 'TS_TEMP', 'EB_TIME', 'EB_TEMP', 'MNY_TIME', 'MNY_TEMP', 'REHO_TIME', 'REHO_TEMP', 'SCR_TIME',
           'SCR_TEMP', 'RPM1', 'RPM2', 'RPM3', 'RPM4', 'RPM5', 'RPM6', 'RPM7', 'RPM8', 'RPM9', 'RPM10', 'RPM11', 'RPM12', 'RPM13',
           'RPM14', 'RPM15', 'RPM16', 'RPM17', 'RPM18', 'RPM19', 'RPM20', 'TEMP1', 'TEMP2', 'TEMP3', 'TEMP4', 'TEMP5', 'TEMP6',
           'TEMP7', 'TEMP8', 'TEMP9', 'TEMP10', 'TEMP11', 'TEMP12', 'TEMP13', 'TEMP14', 'TEMP15', 'TEMP16', 'TEMP17', 'TEMP18',
           'TEMP19', 'TEMP20', 'TIME1', 'TIME2', 'TIME3', 'TIME4', 'TIME5', 'TIME6', 'TIME7', 'TIME8', 'TIME9', 'TIME10', 'TIME11',
           'TIME12', 'TIME13', 'TIME14', 'TIME15', 'TIME16', 'TIME17', 'TIME18', 'TIME19', 'TIME20', 'JUK1', 'JUK2', 'JUK3',
           'JUK4', 'JUK5', 'JUK6', 'JUK7', 'JUK8', 'JUK9', 'JUK10', 'JUK11', 'JUK12', 'JUK13', 'JUK14', 'JUK15', 'JUK16',
           'JUK17', 'JUK18', 'JUK19', 'JUK20', '1_PHR', '2_PHR', '3_PHR', '4_PHR', '5_PHR', '6_PHR', '7_PHR', '8_PHR', '9_PHR',
           '10_PHR', '11_PHR', '12_PHR', '13_PHR', '14_PHR', '15_PHR', '16_PHR', '17_PHR', '18_PHR', '19_PHR', '20_PHR', '21_PHR',
           '1_PUTGB', '2_PUTGB', '3_PUTGB', '4_PUTGB', '5_PUTGB', '6_PUTGB', '7_PUTGB', '8_PUTGB', '9_PUTGB', '10_PUTGB', '11_PUTGB',
           '12_PUTGB', '13_PUTGB', '14_PUTGB', '15_PUTGB', '16_PUTGB', '17_PUTGB', '18_PUTGB', '19_PUTGB', '20_PUTGB', '21_PUTGB',
           'RPM1_c', 'RPM2_c', 'RPM3_c', 'RPM4_c', 'RPM5_c', 'RPM6_c', 'RPM7_c', 'RPM8_c', 'RPM9_c', 'RPM10_c', 'RPM11_c',
           'RPM12_c', 'RPM13_c', 'RPM14_c', 'RPM15_c', 'RPM16_c', 'RPM17_c', 'RPM18_c', 'RPM19_c', 'RPM20_c', 'TEMP1_c', 'TEMP2_c',
           'TEMP3_c', 'TEMP4_c', 'TEMP5_c', 'TEMP6_c', 'TEMP7_c', 'TEMP8_c', 'TEMP9_c', 'TEMP10_c', 'TEMP11_c', 'TEMP12_c',
           'TEMP13_c', 'TEMP14_c', 'TEMP15_c', 'TEMP16_c', 'TEMP17_c', 'TEMP18_c', 'TEMP19_c', 'TEMP20_c', 'TIME1_c', 'TIME2_c',
           'TIME3_c', 'TIME4_c', 'TIME5_c', 'TIME6_c', 'TIME7_c', 'TIME8_c', 'TIME9_c', 'TIME10_c', 'TIME11_c', 'TIME12_c',
           'TIME13_c', 'TIME14_c', 'TIME15_c', 'TIME16_c', 'TIME17_c', 'TIME18_c', 'TIME19_c', 'TIME20_c', 'JUK1_c', 'JUK2_c',
           'JUK3_c', 'JUK4_c', 'JUK5_c', 'JUK6_c', 'JUK7_c', 'JUK8_c', 'JUK9_c', 'JUK10_c', 'JUK11_c', 'JUK12_c', 'JUK13_c',
           'JUK14_c', 'JUK15_c', 'JUK16_c', 'JUK17_c', 'JUK18_c', 'JUK19_c', 'JUK20_c', '1_PHR_c', '2_PHR_c', '3_PHR_c',
           '4_PHR_c', '5_PHR_c', '6_PHR_c', '7_PHR_c', '8_PHR_c', '9_PHR_c', '10_PHR_c', '11_PHR_c', '12_PHR_c', '13_PHR_c',
           '14_PHR_c', '15_PHR_c', '16_PHR_c', '17_PHR_c', '18_PHR_c', '19_PHR_c', '20_PHR_c', '21_PHR_c', '1_PUTGB_c',
           '2_PUTGB_c', '3_PUTGB_c', '4_PUTGB_c', '5_PUTGB_c', '6_PUTGB_c', '7_PUTGB_c', '8_PUTGB_c', '9_PUTGB_c', '10_PUTGB_c',
           '11_PUTGB_c', '12_PUTGB_c', '13_PUTGB_c', '14_PUTGB_c', '15_PUTGB_c', '16_PUTGB_c', '17_PUTGB_c', '18_PUTGB_c', '19_PUTGB_c',
           '20_PUTGB_c', '21_PUTGB_c', 'RUBBER_c', '1_JRCODE_c', '2_JRCODE_c', '3_JRCODE_c', '4_JRCODE_c', '5_JRCODE_c',
           '6_JRCODE_c', '7_JRCODE_c', '8_JRCODE_c', '9_JRCODE_c', '10_JRCODE_c', '11_JRCODE_c', '12_JRCODE_c', '13_JRCODE_c',
           '14_JRCODE_c', '15_JRCODE_c', '16_JRCODE_c', '17_JRCODE_c', '18_JRCODE_c', '19_JRCODE_c', '20_JRCODE_c', '21_JRCODE_c']

# 각 그룹에 해당하는 컬럼명을 포함한 리스트 생성 (X값)
groups = ['RPM', 'TEMP', 'TIME', 'JUK', 'PUTGB', 'PHR', 'JRCODE']

# 제거할 값들 (Y값)
remove_values = ['HS_TEMP', 'SG_TEMP', 'TS_TEMP', 'HS_TIME', 'SG_TIME', 'TS_TIME', 'EB_TIME',  'MNY_TIME','REHO_TIME','SCR_TIME',
                'EB_TEMP','MNY_TEMP','REHO_TEMP','SCR_TEMP']
grouped_columns = {}
for group in groups:
    grouped_columns[group] = [col for col in columns if group in col]

for key in grouped_columns.keys():
    grouped_columns[key] = [value for value in grouped_columns[key] if value not in remove_values]

new_group_dict = {}

for group_name, group_cols in grouped_columns.items():
    non_c_cols = [col for col in group_cols if not col.endswith('_c')]
    c_cols = [col for col in group_cols if col.endswith('_c')]
    new_group_dict[f'{group_name}_non_c'] = non_c_cols
    new_group_dict[f'{group_name}_c'] = c_cols

# 첫번째 정보 제거 (C1)
new_group_dict_ori = new_group_dict    
new_group_dict['JRCODE_non_c'].remove('1_JRCODE')
new_group_dict['PHR_non_c'].remove('1_PHR')
new_group_dict['PUTGB_non_c'].remove('1_PUTGB')

#### 2.2.1) NaN 값 제거 후 리스트 생성
- 원재료코드, PHR, PUTGB

In [13]:
%%time
group_dict = new_group_dict
combined_df = pd.DataFrame()

combined_df['LOTNO'] = final_df['LOTNO']

for group_name, group_cols in group_dict.items():
    tmp_df = final_df[group_cols]
    
    # NaN을 제외하고 리스트로 합치기
    combined_df[f'{group_name}_ls'] = tmp_df.apply(lambda row: [x for x in row if pd.notna(x)], axis=1)
    
    # 리스트의 길이를 새로운 컬럼에 저장 (None 또는 NaN을 제외)
    combined_df[f'{group_name}_num'] = combined_df[f'{group_name}_ls'].apply(lambda x: len(x))

CPU times: user 1min 12s, sys: 516 ms, total: 1min 13s
Wall time: 1min 13s


#### 2.2.2) 리스트에서 0값 제거
- 공정조건 (시간, 온도, 전류, RPM)
- 0값을 제거하고, CMB컬럼 리스트와 FMB 컬럼 리스트를 서로 합치는 작업
- ex) TIME <- [CMD_TIME]+[FMB_TIME]

In [14]:
# 연속된 0을 제거하는 함수
def remove_trailing_zeros(lst):
    while lst and lst[-1] == 0:
        lst.pop()
    return lst

# 각 그룹별로 작업을 수행
for group_name in group_dict.keys():
    # 연속된 0 제거
    combined_df[f'{group_name}_ls'] = combined_df[f'{group_name}_ls'].apply(remove_trailing_zeros)
    
    # 리스트의 길이를 다시 새로운 컬럼에 저장 (None 또는 NaN을 제외)
    combined_df[f'{group_name}_num'] = combined_df[f'{group_name}_ls'].apply(lambda x: len(x))

# 동일한 특성의 CMD, FMB 컬럼 Merge
unique_groups = set(col.split('_')[0] for col in combined_df.columns if '_ls' in col)

for group in unique_groups:
    ls_cols = sorted([col for col in combined_df.columns if group in col and '_ls' in col], key=lambda x: '_non_c' in x)
    combined_df[f'{group}_combined_ls'] = combined_df[ls_cols].apply(lambda row: [x for sublist in row for x in sublist], axis=1)
    # 리스트의 길이를 새로운 컬럼에 저장 (None 또는 NaN을 제외)
    combined_df[f'{group}_combined_num'] = combined_df[f'{group}_combined_ls'].apply(lambda x: len(x) if x is not None else 0)
    
    
# 전처리 된 리스트 형태의 데이터를 실제 리스트 형식의 데이터로 변경
# 사용할 컬럼
filter_df = combined_df[['TEMP_combined_ls', 'PHR_combined_ls', 'PUTGB_combined_ls', 'RPM_combined_ls', 'TIME_combined_ls',
                         'JRCODE_combined_ls', 'JUK_combined_ls']]

import ast
# 문자열이 리스트 형태인지 확인하는 함수 정의
def is_list_like(s):
    try:
        ast.literal_eval(s)
        return True
    except (ValueError, SyntaxError):
        return False

# 문자열을 리스트로 변환하는 함수 정의
def convert_to_list(s):
    if is_list_like(s):
        return ast.literal_eval(s)
    return s

# 문자열을 리스트로 변환
for col in filter_df.columns:
    filter_df[col] = filter_df[col].apply(convert_to_list)   

#### 2.2.3) CMB컬럼+FMB컬럼으로 합쳐진 리스트 패딩작업

In [27]:
# 변수 특성에 따라 집합 생성
process_cols = ['TEMP_combined_ls', 'TIME_combined_ls', 'RPM_combined_ls', 'JUK_combined_ls']
integration_cols = ['PUTGB_combined_ls', 'PHR_combined_ls', 'JRCODE_combined_ls']

process_col_max = max([max(filter_df[col].apply(lambda x: len(x) if isinstance(x, list) else 0)) for col in process_cols if col.endswith('_ls')])
integration_col_max = max([max(filter_df[col].apply(lambda x: len(x) if isinstance(x, list) else 0)) for col in integration_cols if col.endswith('_ls')])

# 기존에 계산된 최대 길이를 사용
for col in process_cols:
    max_len = process_col_max
    filter_df[col] = filter_df[col].apply(lambda x: x + [np.nan] * (max_len - len(x)) 
                                          if isinstance(x, list) else x)

for col in integration_cols:
    max_len = integration_col_max
    filter_df[col] = filter_df[col].apply(lambda x: x + [np.nan] * (max_len - len(x)) 
                                          if isinstance(x, list) else x)

# 리스트 확장
expanded_cols = process_cols + integration_cols
filter_df_expanded = pd.concat([filter_df.drop(expanded_cols, axis=1)] +
                               [pd.DataFrame(filter_df[col].to_list(), 
                                             columns=[f"{col}_{i+1}" for i in range(max(filter_df[col].apply(lambda x: len(x) if isinstance(x, list) else 0)))])
                                for col in expanded_cols], axis=1)

##### 패딩작업이 완료된 X(공정조건, 재료코드 리스트)데이터와 Y(물성, 불량여부) 데이터 Merge
- 불필요한 컬럼들이 제거된 최적화된 최종 데이터셋으로 봐도 무방

In [65]:
#데이터 프레임으로 되어 있는 데이터셋에서 전처리된 리스트으로 들어가야하는 컬럼들(공정조건) 제거
# 공정조건 컬럼리스트 정의
flattened_list_one_liner = [item for sublist in new_group_dict_ori.values() for item in sublist]

# 공정조건별 컬럼 제거
final_df_filter = final_df.drop(flattened_list_one_liner, axis=1)

# 추가적으로 필요없거나 중복된 컬럼 제거
temp = final_df_filter.drop(['RUBBER_c','1_JRCODE','1_PHR','1_PUTGB'], axis=1)

col_names_dict_1 = {
    'RPM_combined_ls': ['RPM1','RPM2','RPM3','RPM4','RPM5','RPM6','RPM7','RPM8','RPM9','RPM10','RPM11','RPM12','RPM13','RPM14','RPM15','RPM16','RPM17','RPM18','RPM19','RPM20'],
    'TEMP_combined_ls': ['TEMP1','TEMP2','TEMP3','TEMP4','TEMP5','TEMP6','TEMP7','TEMP8','TEMP9','TEMP10','TEMP11','TEMP12','TEMP13','TEMP14','TEMP15','TEMP16','TEMP17','TEMP18','TEMP19','TEMP20'],
    'TIME_combined_ls': ['TIME1','TIME2','TIME3','TIME4','TIME5','TIME6','TIME7','TIME8','TIME9','TIME10','TIME11','TIME12','TIME13','TIME14','TIME15','TIME16','TIME17','TIME18','TIME19','TIME20'],
    'JUK_combined_ls': ['JUK1','JUK2','JUK3','JUK4','JUK5','JUK6','JUK7','JUK8','JUK9','JUK10','JUK11','JUK12','JUK13','JUK14','JUK15','JUK16','JUK17','JUK18','JUK19','JUK20'],
}
col_names_dict_2 = {
    'JRCODE_combined_ls': ['1_JRCODE','2_JRCODE','3_JRCODE','4_JRCODE','5_JRCODE','6_JRCODE','7_JRCODE','8_JRCODE','9_JRCODE','10_JRCODE','11_JRCODE','12_JRCODE','13_JRCODE','14_JRCODE','15_JRCODE','16_JRCODE','17_JRCODE','18_JRCODE','19_JRCODE','20_JRCODE','21_JRCODE'],
    'PHR_combined_ls': ['1_PHR','2_PHR','3_PHR','4_PHR','5_PHR','6_PHR','7_PHR','8_PHR','9_PHR','10_PHR','11_PHR','12_PHR','13_PHR','14_PHR','15_PHR','16_PHR','17_PHR','18_PHR','19_PHR','20_PHR','21_PHR'],
    'PUTGB_combined_ls': ['1_PUTGB','2_PUTGB','3_PUTGB','4_PUTGB','5_PUTGB','6_PUTGB','7_PUTGB','8_PUTGB','9_PUTGB','10_PUTGB','11_PUTGB','12_PUTGB','13_PUTGB','14_PUTGB','15_PUTGB','16_PUTGB','17_PUTGB','18_PUTGB','19_PUTGB','20_PUTGB','21_PUTGB']
}

for prefix, new_cols in col_names_dict_1.items():
    old_cols = [col for col in filter_df_expanded.columns if col.startswith(prefix)]
    
    if len(old_cols) > len(new_cols):
        base_name = new_cols[0].rstrip('1234567890')
        extra_cols = [f"{base_name}{i}" for i in range(len(new_cols) + 1, len(old_cols) + 1)]
        new_cols.extend(extra_cols)
    
    rename_dict = {old_cols[i]: new_cols[i] for i in range(len(old_cols))}
    filter_df_expanded.rename(columns=rename_dict, inplace=True)
    
for prefix, new_cols in col_names_dict_2.items():
    old_cols = [col for col in filter_df_expanded.columns if col.startswith(prefix)]
    
    if len(old_cols) > len(new_cols):
        base_name = new_cols[0].rstrip('1234567890')
        extra_cols = [f"{i}_{base_name.split('_')[1]}" for i in range(len(new_cols) + 1, len(old_cols) + 1)]
        new_cols.extend(extra_cols)
    
    rename_dict = {old_cols[i]: new_cols[i] for i in range(len(old_cols))}
    filter_df_expanded.rename(columns=rename_dict, inplace=True)

    
# X, Y 데이터셋 Merge
# temp -> Y데이터
# filter_df_expanded -> 최종적으로 전처리된 X데이터

filter_df_expanded_pre = pd.concat([temp, filter_df_expanded], axis=1)

## 3. 최종적으로 완성된 CMB와 연계된 FMB 데이터셋 전처리
- RESULT 컬럼 전처리
- TIME 컬럼 이상치 처리
- PHR 이상치 처리 및 상대적 비중으로 조정

### 3.1 불량여부(RESULT라는 단어가 들어간) 컬럼에 대해 P,F 전처리
- P->1, F->0

In [58]:
# 'RESULT' 문자열이 들어가는 컬럼만 선택
result_columns = [col for col in filter_df_expanded_pre.columns if 'RESULT' in col]

# P는 1로, F는 0으로 변환
for col in result_columns:
    filter_df_expanded_pre[col] = filter_df_expanded_pre[col].map({'P': 1, 'F': 0})

### 3.2 검사시간(TIME이라는 변수가 들어간 컬럼)에 대해 이상치(연산자) 전처리
- ex) 5+9->14

In [59]:
import re

time_columns = [col for col in filter_df_expanded_pre.columns if 'TIME' in col]

# 연산자 적용 함수
def apply_operator(expression):
    if isinstance(expression, (int, float)):
        return expression
    
    if re.match("^[0-9+\-*/]+$", expression):
        try:
            return eval(expression)
        except ZeroDivisionError:
            return "Division by zero"
        except Exception as e:
            return str(e)
    else:
        return expression

for col in time_columns:
    filter_df_expanded_pre[col] = filter_df_expanded_pre[col].apply(apply_operator)

### 3.3 음의 PHR 값을 갖는 배합 데이터 제거

In [ ]:
tmp_df = filter_df_expanded_pre.copy()

for col in [f'{i}_PHR' for i in range(1, 26)]:
    tmp_df = tmp_df[(tmp_df[col]>=0)|(tmp_df[col].isna())]
print(f"before data num: {len(filter_df_expanded_pre)}, after data num: {len(tmp_df)}")

filter_df_expanded_pre = tmp_df.copy()

before data num: 232901, after data num: 232819


### 3.4 CMB와 연계된 FMB 데이터에 대한 PHR 상대값 조절
- 각 행별 PHR 특성 컬럼들의 최대값을 100으로 환산하여 전처리

In [ ]:
%%time
phr_cols = [f'{i}_PHR' for i in range(1, 26)]
filter_df_expanded_pre[phr_cols] = filter_df_expanded_pre[phr_cols].apply(lambda x: x / x.max() * 100, axis=1)

CPU times: total: 34.8 s
Wall time: 34.9 s


## 4. 예측 대상별 회귀(물성값)/분류(불량여부) 데이터셋 구축

### 4.1 데이터셋 생성

In [61]:
basic_cols = ['LOTNO','RUBBER']
# 변수명 조정 필요
# ========================================================
rpm_cols = ['RPM1','RPM2','RPM3','RPM4','RPM5','RPM6','RPM7','RPM8','RPM9','RPM10','RPM11','RPM12','RPM13','RPM14','RPM15','RPM16','RPM17','RPM18','RPM19','RPM20','RPM21','RPM22','RPM23']
temp_cols = ['TEMP1','TEMP2','TEMP3','TEMP4','TEMP5','TEMP6','TEMP7','TEMP8','TEMP9','TEMP10','TEMP11','TEMP12','TEMP13','TEMP14','TEMP15','TEMP16','TEMP17','TEMP18','TEMP19','TEMP20','TEMP21','TEMP22','TEMP23']
time_cols = ['TIME1','TIME2','TIME3','TIME4','TIME5','TIME6','TIME7','TIME8','TIME9','TIME10','TIME11','TIME12','TIME13','TIME14','TIME15','TIME16','TIME17','TIME18','TIME19','TIME20','TIME21','TIME22','TIME23']
juk_cols = ['JUK1','JUK2','JUK3','JUK4','JUK5','JUK6','JUK7','JUK8','JUK9','JUK10','JUK11','JUK12','JUK13','JUK14','JUK15','JUK16','JUK17','JUK18','JUK19','JUK20','JUK21','JUK22','JUK23']
jrcode_cols = ['1_JRCODE','2_JRCODE','3_JRCODE','4_JRCODE','5_JRCODE','6_JRCODE','7_JRCODE','8_JRCODE','9_JRCODE','10_JRCODE','11_JRCODE','12_JRCODE','13_JRCODE','14_JRCODE','15_JRCODE','16_JRCODE','17_JRCODE','18_JRCODE','19_JRCODE','20_JRCODE','21_JRCODE','22_JRCODE','23_JRCODE','24_JRCODE','25_JRCODE']
phr_cols = ['1_PHR','2_PHR','3_PHR','4_PHR','5_PHR','6_PHR','7_PHR','8_PHR','9_PHR','10_PHR','11_PHR','12_PHR','13_PHR','14_PHR','15_PHR','16_PHR','17_PHR','18_PHR','19_PHR','20_PHR','21_PHR','22_PHR','23_PHR','24_PHR','25_PHR']
putgb_cols = ['1_PUTGB','2_PUTGB','3_PUTGB','4_PUTGB','5_PUTGB','6_PUTGB','7_PUTGB','8_PUTGB','9_PUTGB','10_PUTGB','11_PUTGB','12_PUTGB','13_PUTGB','14_PUTGB','15_PUTGB','16_PUTGB','17_PUTGB','18_PUTGB','19_PUTGB','20_PUTGB','21_PUTGB','22_PUTGB','23_PUTGB','24_PUTGB','25_PUTGB']
# 변수명 조정 필요
# ========================================================

trg_ispt_cols = ['HS_TIME','SG_TIME','TS_TIME','EB_TIME','MNY_TIME','REHO_TIME','SCR_TIME','HS_TEMP','SG_TEMP','TS_TEMP','EB_TEMP','MNY_TEMP','REHO_TEMP','SCR_TEMP']
trg_regression_cols = ['HS','SG','TS','EB','MNY','REHO_MIN','REHO_MAX','REHO_TS2','REHO_TC90','SCR']
trg_classification_cols = ['HS_RESULT','SG_RESULT','TS_RESULT','EB_RESULT','MNY_RESULT','REHO_RESULT','SCR_RESULT']

data_df = filter_df_expanded_pre.copy()

show_df = pd.DataFrame(columns=['colNm','data#','col#'])

for yCol in trg_regression_cols:
    print(yCol)
    # 각 예측 대상별 검사 시간과 조건 변수명 리스트 생성
    tmp_ispt_cols = [col for col in trg_ispt_cols if yCol[:2] in col]
    # 데이터셋 구축을 위한 변수 리스트에 따른 데이터프레임 컬럼 필터링
    fin_df = data_df[basic_cols+rpm_cols+temp_cols+time_cols+juk_cols+jrcode_cols+phr_cols+putgb_cols+tmp_ispt_cols+[yCol]]
    # 예측 대상이 되는 값이 nan인 경우 제외
    fin_df = fin_df[~(fin_df[yCol].isna())&(fin_df[yCol]!=0)]
    fin_df.to_csv(f'./tmp_dataset/mes_regr_fmb_data_{yCol}_fin_noZero.csv', index=False)
    show_df.loc[len(show_df)] = [yCol, len(fin_df), len(fin_df.columns)]

for yCol in trg_classification_cols:
    print(yCol)
    # 각 예측 대상별 검사 시간과 조건 변수명 리스트 생성
    tmp_ispt_cols = [col for col in trg_ispt_cols if yCol[:2] in col]
    # 데이터셋 구축을 위한 변수 리스트에 따른 데이터프레임 컬럼 필터링
    fin_df = data_df[basic_cols+rpm_cols+temp_cols+time_cols+juk_cols+jrcode_cols+phr_cols+putgb_cols+tmp_ispt_cols+[yCol]]
    # 예측 대상이 되는 값이 nan인 경우 제외
    fin_df = fin_df[~(fin_df[yCol].isna())]
    fin_df.to_csv(f'./tmp_dataset/mes_clas_fmb_data_{yCol}_fin.csv', index=False)
    show_df.loc[len(show_df)] = [yCol, len(fin_df), len(fin_df.columns)]

HS
SG
TS
EB
MNY
REHO_MIN
REHO_MAX
REHO_TS2
REHO_TC90
SCR
HS_RESULT
SG_RESULT
TS_RESULT
EB_RESULT
MNY_RESULT
REHO_RESULT
SCR_RESULT
